In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce

In [2]:
spark = SparkSession.builder.appName("Create_FIPEZAP").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/02 12:31:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

df_fipezap = spark.read.option("delimiter", ",").option("header", True).csv("dados_fipezap.csv")\
.withColumn("DT_REG_IC", F.lit(F.to_date(F.col("DT_REG_IC"), "yyyy-MM-dd")))\
.withColumn("VL_IC", F.round(F.col("VL_IC"), 2))\
.withColumn("VL_VRC_IC", F.round(F.col("VL_VRC_IC"), 2))\
.withColumn("DT_REF", F.lit(F.to_date(F.col("DT_REF"), "yyyy-MM-dd")))

df_fipezap = df_fipezap\
.withColumn("VL_VRC_IC", F.when(F.col("VL_VRC_IC").isNotNull(), F.col("VL_VRC_IC")).otherwise(0.00))\
.withColumn("VL_IC", F.col("VL_IC").cast("float"))\
.withColumn("VL_VRC_IC", F.col("VL_VRC_IC").cast("float"))

df_fipezap.write.parquet("indices_mercado_externo.parquet", mode="overwrite")

df_fipezap.orderBy("DT_REG_IC", ascending=False).show(truncate=False)

+----------+-------+------+---------+----------+
|DT_REG_IC |NM_IC  |VL_IC |VL_VRC_IC|DT_REF    |
+----------+-------+------+---------+----------+
|2023-04-01|FIPEZAP|151.96|0.45     |2024-06-02|
|2023-03-01|FIPEZAP|151.29|0.42     |2024-06-02|
|2023-02-01|FIPEZAP|150.65|0.38     |2024-06-02|
|2023-01-01|FIPEZAP|150.08|0.3      |2024-06-02|
|2022-12-01|FIPEZAP|149.64|0.3      |2024-06-02|
|2022-11-01|FIPEZAP|149.18|0.46     |2024-06-02|
|2022-10-01|FIPEZAP|148.5 |0.59     |2024-06-02|
|2022-09-01|FIPEZAP|147.63|0.6      |2024-06-02|
|2022-08-01|FIPEZAP|146.75|0.6      |2024-06-02|
|2022-07-01|FIPEZAP|145.88|0.52     |2024-06-02|
|2022-06-01|FIPEZAP|145.13|0.47     |2024-06-02|
|2022-05-01|FIPEZAP|144.46|0.41     |2024-06-02|
|2022-04-01|FIPEZAP|143.87|0.48     |2024-06-02|
|2022-03-01|FIPEZAP|143.18|0.55     |2024-06-02|
|2022-02-01|FIPEZAP|142.4 |0.49     |2024-06-02|
|2022-01-01|FIPEZAP|141.7 |0.53     |2024-06-02|
|2021-12-01|FIPEZAP|140.96|0.48     |2024-06-02|
|2021-11-01|FIPEZAP|

In [4]:
df_fipezap.printSchema()

root
 |-- DT_REG_IC: date (nullable = true)
 |-- NM_IC: string (nullable = true)
 |-- VL_IC: float (nullable = true)
 |-- VL_VRC_IC: float (nullable = true)
 |-- DT_REF: date (nullable = true)



In [5]:
df_csv = df_fipezap
df_base = spark.read.parquet("indices_mercado_externo.parquet")

In [6]:
df_base.printSchema()

root
 |-- DT_REG_IC: date (nullable = true)
 |-- NM_IC: string (nullable = true)
 |-- VL_IC: float (nullable = true)
 |-- VL_VRC_IC: float (nullable = true)
 |-- DT_REF: date (nullable = true)



In [7]:
df_base.orderBy("DT_REG_IC", ascending=False).show(truncate=False)

+----------+-------+------+---------+----------+
|DT_REG_IC |NM_IC  |VL_IC |VL_VRC_IC|DT_REF    |
+----------+-------+------+---------+----------+
|2023-04-01|FIPEZAP|151.96|0.45     |2024-06-02|
|2023-03-01|FIPEZAP|151.29|0.42     |2024-06-02|
|2023-02-01|FIPEZAP|150.65|0.38     |2024-06-02|
|2023-01-01|FIPEZAP|150.08|0.3      |2024-06-02|
|2022-12-01|FIPEZAP|149.64|0.3      |2024-06-02|
|2022-11-01|FIPEZAP|149.18|0.46     |2024-06-02|
|2022-10-01|FIPEZAP|148.5 |0.59     |2024-06-02|
|2022-09-01|FIPEZAP|147.63|0.6      |2024-06-02|
|2022-08-01|FIPEZAP|146.75|0.6      |2024-06-02|
|2022-07-01|FIPEZAP|145.88|0.52     |2024-06-02|
|2022-06-01|FIPEZAP|145.13|0.47     |2024-06-02|
|2022-05-01|FIPEZAP|144.46|0.41     |2024-06-02|
|2022-04-01|FIPEZAP|143.87|0.48     |2024-06-02|
|2022-03-01|FIPEZAP|143.18|0.55     |2024-06-02|
|2022-02-01|FIPEZAP|142.4 |0.49     |2024-06-02|
|2022-01-01|FIPEZAP|141.7 |0.53     |2024-06-02|
|2021-12-01|FIPEZAP|140.96|0.48     |2024-06-02|
|2021-11-01|FIPEZAP|